<a href="https://colab.research.google.com/github/rafa-cc/Proyecto-Final-R/blob/main/Proyecto_Final_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Análisis de Productos de Amazon__
__💬 Descripción:__

> En este proyecto se usan datos sobre productos de comestibles de Amazon. Se cuenta con columnas como cifras de ventas, ingresos, dimensiones del producto, calificaciones y otras métricas esenciales.

__🎯 Objetivos:__
> - Clasificar los productos en categorías de tamaño basado en sus dimensiones.
- Proponer dimensiones para el empaquetado más óptimo en cada categoría.
- Probar si la calificación promedio de los clientes afecta las ventas.
- Examinar cómo se relaciona el precio con otras variables.
- Analizar la estacionalidad de las ventas.
- Clasificar los productos en categorías de alto, medio o bajo rendimiento basado en sus características.



El trabajo está divido en las siguientes secciones:

◉ _Librerías_  ◉ _Importación_  ◉ _Análisis Exploratorio (EDA)_  ◉ _Ingeniería de Variables_  ◉ _Análisis de Empaquetado_

## __Librerías__

In [2]:
install.packages("e1071")
library(dplyr)
library(ggplot2)
library(e1071)
separate <- tidyr::separate # Se usa para la separación de la columna 'Dimensions'

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘proxy’



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## __Importación__

💡 El data set se encuentra [aquí](https://github.com/rafa-cc/Proyecto-Final-R/blob/main/dataset.csv)

In [3]:
# El dataset se encuentra en el repositorio del proyecto
# Parece que el link se actualiza, volver a copiarlo si es necesario
df_raw <- read.csv("https://raw.githubusercontent.com/rafa-cc/Proyecto-Final-R/main/dataset.csv")

## __Análisis Exploratorio__

In [4]:
head(df_raw)

,Product.Details,ASIN,Brand,Price,Sales,Revenue,BSR,FBA.Fees,Active.Sellers..,Ratings,Review.Count,Images,Review.velocity,Buy.Box,Category,Size.Tier,Delivery,Dimensions,Weight,Creation.Date
,<chr>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,"Lindt Lindor Milk Chocolate Truffles Box - The Ideal Gift - Chocolate Balls with a Smooth Melting Filling, 200 g",B00NW479QO,Lindt,3.50,13466,47131,3,2.62,30,5,44069,12,868,Amazon,Food Cupboard,Small Oversize,AMZ,4.2x6.3x3.1,0.51,9/26/2014
2,"Andrex Toilet Roll - Gentle Clean Toilet Paper, 45 Toilet Rolls",B004OCO20E,Andrex,18.28,13338,243819,2,8.38,30,5,40397,5,830,Amazon,Grocery,Large Oversize,AMZ,13.5x21.8x13.0,9.77,2/17/2011
3,Baileys Original Irish Cream Liqueur 1L,B0049NYI7K,BAILEYS,9.99,11194,111828,4,3.93,22,5,3827,6,235,Amazon,"Beer, Wine & Spirits",Large Oversize,AMZ,3.7x12.2x3.7,3.73,10/21/2010
4,"Tassimo Kenco Americano Grande Coffee Pods (Pack of 5, Total 80 Coffee Capsules)",B016DEGFWI,Tassimo,14.87,13492,200626,2,5.19,11,5,28800,5,466,Amazon,Fresh & Chilled,Large Oversize,AMZ,6.8x15.5x6.2,3.26,8/4/2010
5,"Tassimo Costa Gingerbread Latte Coffee Pods (Pack of 5, Total 40 Coffee Capsules)",B07JVF2FCV,Tassimo,45.00,13377,601965,6,11.03,1,5,27494,9,223,Bountyville,Drinks,Large Oversize,FBA,6.7x15.4x6.1,4.74,9/3/2018
6,"Regina XXL Kitchen Roll, 8 Rolls, 600 Extra Large Sheets",B07T2SCHM7,Regina,8.00,11983,95864,5,6.16,21,5,11120,6,701,,Grocery,Large Oversize,MFN,10.6x34.6x5.1,5.34,5/1/2020


In [5]:
names(df_raw)

[1] "Product.Details"  "ASIN"             "Brand"            "Price"           
 [5] "Sales"            "Revenue"          "BSR"              "FBA.Fees"        
 [9] "Active.Sellers.." "Ratings"          "Review.Count"     "Images"          
[13] "Review.velocity"  "Buy.Box"          "Category"         "Size.Tier"       
[17] "Delivery"         "Dimensions"       "Weight"           "Creation.Date"

In [6]:
# Descartar las columnas que no son de interés y dividir la columna 'Dimensions' en 'Length', 'Width' y 'Height'

df <- select(df_raw,-Brand, -ASIN, -Review.velocity, -Images, -Buy.Box, -Size.Tier, -Delivery, -Weight, -FBA.Fees) %>%
      separate(Dimensions, into = c("Length", "Width", "Height"), sep = "x", convert = TRUE)
names(df)

Warning message:
“Expected 3 pieces. Missing pieces filled with `NA` in 1516 rows [51, 62, 66,
99, 114, 131, 147, 191, 203, 209, 212, 213, 218, 226, 227, 229, 231, 232, 234,
235, ...].”


[1] "Product.Details"  "Price"            "Sales"            "Revenue"         
 [5] "BSR"              "Active.Sellers.." "Ratings"          "Review.Count"    
 [9] "Category"         "Length"           "Width"            "Height"          
[13] "Creation.Date"

In [7]:
# Contar los datos nulos en cada columna
sapply(df, function(x) sum(is.na(x)))

Product.Details            Price            Sales          Revenue 
               0             1264             1802             1511 
             BSR Active.Sellers..          Ratings     Review.Count 
             453               51              828              828 
        Category           Length            Width           Height 
               0             1516             1516             1516 
   Creation.Date 
               0

In [8]:
# Nuevo dataframe que solo incluye las filas sin valores nulos en las columnas 'Price' y 'Sales'
df_sin_nulos <- df[complete.cases(df$Price, df$Sales), ]

# Aún hay datos nulos pero pueden descartarse en otro momento específico
sapply(df_sin_nulos, function(x) sum(is.na(x)))

Product.Details            Price            Sales          Revenue 
               0                0                0                0 
             BSR Active.Sellers..          Ratings     Review.Count 
               0               36              238              238 
        Category           Length            Width           Height 
               0              667              667              667 
   Creation.Date 
               0

In [9]:
# Casting de 'Creation.Date'
df_sin_nulos$Creation.Date <- as.Date(df_sin_nulos$Creation.Date, format = '%d/%m/%Y')

# Factor de 'Ratings'
df_sin_nulos$Ratings <- factor(df_sin_nulos$Ratings, ordered = TRUE)

summary(df_sin_nulos)

 Product.Details        Price            Sales            Revenue        
 Length:4242        Min.   :  0.50   Min.   :    0.0   Min.   :     0.0  
 Class :character   1st Qu.:  7.98   1st Qu.:   14.0   1st Qu.:   198.2  
 Mode  :character   Median : 12.99   Median :  114.5   Median :  1303.0  
                    Mean   : 15.09   Mean   : 1046.5   Mean   : 12900.9  
                    3rd Qu.: 19.99   3rd Qu.:  619.0   3rd Qu.:  6876.0  
                    Max.   :254.99   Max.   :23288.0   Max.   :601965.0  
                                                                         
      BSR          Active.Sellers..  Ratings      Review.Count    
 Min.   :      0   Min.   :  1.000   1   :  10   Min.   :    1.0  
 1st Qu.:   2896   1st Qu.:  1.000   2   :   8   1st Qu.:   23.0  
 Median :  11652   Median :  2.000   3   :  67   Median :  168.5  
 Mean   :  38034   Mean   :  4.842   4   :1175   Mean   : 1157.0  
 3rd Qu.:  58930   3rd Qu.:  5.000   5   :2744   3rd Qu.:  680.2  
 Max. 

## __Ingeniería de Variables__

In [10]:
# Valores únicos de 'Category'
print(unique(df_sin_nulos$Category))

 [1] "Food Cupboard"                    "Grocery"                         
 [3] "Beer, Wine & Spirits"             "Fresh & Chilled"                 
 [5] "Drinks"                           "Home & Kitchen"                  
 [7] "Business, Industry & Science"     "Automotive"                      
 [9] "Hampers & Gourmet Gifts"          "Health & Personal Care"          
[11] "Outdoor Recreation"               "Cooking & Dining"                
[13] "Diet & Nutrition"                 "Home Accessories"                
[15] "Vitamins, Minerals & Supplements" "Frozen"                          
[17] "Bakery"                           "Garden & Outdoors"               
[19] "Toys & Games"                     "Beauty"                          


Estas categorias podrías simplificarse tentativamente en la siguiente forma:

> | Category | Subcategories |
|---|---|
| **Alimentos y despensa** |   ◉ Food Cupboard  ◉ Grocery  ◉ Fresh & Chilled   ◉ Cooking & Dining   ◉ Frozen   ◉ Bakery |
| **Bebidas** |   ◉ Beer, Wine & Spirits   ◉ Drinks |
| **Hogar y cocina** |   ◉ Home & Kitchen   ◉ Home Accessories   ◉ Garden & Outdoors |
| **Cuidado personal y salud** |   ◉ Health & Personal Care   ◉ Beauty   ◉ Vitamins, Minerals & Supplements   ◉ Diet & Nutrition |
| **Regalos y ocio** |   ◉ Hampers & Gourmet Gifts   ◉ Outdoor Recreation   ◉ Toys & Games |
| **Industria y ciencia** |   ◉ Business, Industry & Science   ◉ Automotive |






In [11]:
# Definir las categorías de agrupación
categorias <- list(
  Food_Grocery = c("Food Cupboard", "Grocery", "Fresh & Chilled", "Cooking & Dining", "Frozen", "Bakery"),
  Drinks = c("Beer, Wine & Spirits", "Drinks"),
  Home_Kitchen = c("Home & Kitchen", "Home Accessories", "Garden & Outdoors"),
  Health_Care = c("Health & Personal Care", "Beauty", "Vitamins, Minerals & Supplements", "Diet & Nutrition"),
  Gifts_Entertainment = c("Hampers & Gourmet Gifts", "Outdoor Recreation", "Toys & Games"),
  Industry_Science = c("Business, Industry & Science", "Automotive")
)

# Función para asignar la categoría madre
asignar_categoria <- function(cat) {
  for (nombre_categoria in names(categorias)) {
    if (cat %in% categorias[[nombre_categoria]]) {
      return(nombre_categoria)
    }
  }
}

# Crear la nueva columna y hacerla factor, desechar 'Category' pues ya no es necesaria
df_main <- df_sin_nulos %>%
  mutate(High.Category = sapply(Category, asignar_categoria)) %>%
  mutate(High.Category = factor(High.Category)) %>%
  select(-Category)

summary(df_main)

 Product.Details        Price            Sales            Revenue        
 Length:4242        Min.   :  0.50   Min.   :    0.0   Min.   :     0.0  
 Class :character   1st Qu.:  7.98   1st Qu.:   14.0   1st Qu.:   198.2  
 Mode  :character   Median : 12.99   Median :  114.5   Median :  1303.0  
                    Mean   : 15.09   Mean   : 1046.5   Mean   : 12900.9  
                    3rd Qu.: 19.99   3rd Qu.:  619.0   3rd Qu.:  6876.0  
                    Max.   :254.99   Max.   :23288.0   Max.   :601965.0  
                                                                         
      BSR          Active.Sellers..  Ratings      Review.Count    
 Min.   :      0   Min.   :  1.000   1   :  10   Min.   :    1.0  
 1st Qu.:   2896   1st Qu.:  1.000   2   :   8   1st Qu.:   23.0  
 Median :  11652   Median :  2.000   3   :  67   Median :  168.5  
 Mean   :  38034   Mean   :  4.842   4   :1175   Mean   : 1157.0  
 3rd Qu.:  58930   3rd Qu.:  5.000   5   :2744   3rd Qu.:  680.2  
 Max. 

## __Análisis de Empaquetado__

Se comienza ordenando cada fila de menor a mayor, no importa si se mezaclan las columnas pues las mediciones de Length, Width, Height son subjetivas dependiendo de la orientación a la hora de medir.

In [20]:
# Nuevo dataframe que solo incluye las filas sin valores nulos en las columnas de dimensiones
df_empaquetado <- df_main[complete.cases(df_main$Length), ]

# En cada fila ordenar las columnas de dimensiones de menor a mayor
df_temp <- df_empaquetado %>%
        select(Length, Width, Height) %>%
        apply(1, sort) %>%
        t() %>%
        as.data.frame() %>%
        setNames(c("Length", "Width", "Height"))

# Reasignar los valores ordenados y se ordenan las filas de menos a mayor basado en 'Length'
df_empaquetado <- df_empaquetado %>%
        select(Length, Width, Height) %>%
        mutate(Length = df_temp$Length, Width = df_temp$Width, Height = df_temp$Height) %>%
        arrange(Length) %>%
        slice(-1) # Eliminar el primer registro pues sus dimensiones son 0's
# Reiniciar los indices después de los reordenamientos
row.names(df_empaquetado) <- NULL

# Guardar el dataframe para su análisis con Python
write.csv(x=df_empaquetado, file="data.csv", col.names = TRUE, row.names = FALSE)

Warning message in write.csv(x = df_empaquetado, file = "data.csv", col.names = TRUE, :
“attempt to set 'col.names' ignored”


### __Análisis con Python__

A partir de aquí se decidió hacer una clasificación de los posibles tipos de caja usando un modelo de maquina de soporte vectorial en Python. Conviene clasificar los empaques por la forma que tienen

Basado en el razonamiento, los posibles tipos de caja propuesto fueron los siguientes:

|Pizza|Cubo|Leche|Estaca|Tabla|Desconocido|
|---|---|---|---|---|---|
|2 dimensiones muy parecidas con <br> lo que se tiene una base aproxima-<br>damente  cuadrada y la 3er dimension <br> es más pequeña respecto a las otras|las 3 medidas son muy similares <br> entre si, generando una forma <br>cercana a la de un cubo| 3 dimensiones que están <br> aproximadamente equidistantes <br> entre sí|2 dimensiones muy parecidas con <br> lo que se tiene una base aproxima-<br>damente  cuadrada y la 3er dimension <br> es más grande respecto a las otras| 2 dimensiones diferentes, y <br> la tercera es más pequeña en <br> comparación con las otras dos| Cualquiera que no sea una <br> de las anteriores|

Para ver las referencias visuales estas formas consultar el [repositorio](https://github.com/rafa-cc/Proyecto-Final-R/tree/main/Imagenes)

Para ver el análisis de Python consultar el siguiente [archivo](https://colab.research.google.com/drive/1pnK2Lhk898ryd-9OdGD4vEEa8AAO1Pp5?usp=sharing)